In [1]:
import json
import csv
import numpy as np
import pandas as pd
from os import path, mkdir
import glob
from datetime import datetime, timedelta, date
#pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# Load Data

In [ ]:
def parse_dt(x):
    h,m=map(int,x[0].split(':'))
    ds = x[1]
    hours_add=11 if (ds>=datetime(2019,3,31,0,0,0) and ds<datetime(2019,10,27,0,0,0)) or (ds>=datetime(2018,3,25,0,0,0) and ds<datetime(2018,10,28,0,0,0)) or (ds>=datetime(2017,3,26,0,0,0) and ds<datetime(2017,10,29,0,0,0)) or (ds>=datetime(2016,3,27,0,0,0) and ds<datetime(2016,10,30,0,0,0)) or ds<datetime(2015,10,25,0,0,0) else 12
    if h<9:
        h+=hours_add
    
    ds+= timedelta(hours=h)
    ds+= timedelta(minutes=m)
    return ds

allhorses=pd.read_csv('data/horses_bf.csv', index_col=False)
# an index from horseName field
allhorses['wname']=allhorses['horseName'].str.lower().replace(regex=True,to_replace=r'\W',value=r'')

dic=['DSQ','PU', 'F', 'UR', 'RR', 'RO', 'SU', 'REF', 'BD', 'CO', 'LFT', '0']
allhorses.loc[allhorses['position'].isin(dic),'position']='40'
allhorses['position']=allhorses['position'].astype(int)

horses=allhorses[allhorses['position']==1]
horses['wname']=horses['wname'].str.lower().replace(regex=True,to_replace=r'\W',value=r'')

allraces=pd.read_csv('data/races_bf.csv', index_col=False)
allraces['countryCode']=allraces['course'].apply(lambda x: x.split('(')[-1].replace(')',''))
allraces['countryCode']=np.where(allraces['countryCode'].str.len()>4,'GB',allraces['countryCode'])

dic={'AW':'GB', 'IRE':'IE', 'USA':'US','AUS':'AU','Ayr':'GB','JPN':'JP','UAE':'AE','ITY':'IT'}
allraces['countryCode']=allraces['countryCode'].replace(dic)

allraces['date']=pd.to_datetime(allraces['date'])
allraces['marketTime']=allraces.apply(lambda x: parse_dt(x[['time','date']]), axis=1)
allraces=allraces.merge(horses[['rid','wname']], on='rid', how='left')

allhorses=allhorses.merge(allraces[['rid','marketTime']], on='rid', how='left')

In [ ]:
cols_races=['rid','course','time','date','marketTime','title','runners','rclass','band','ages','distance','condition','hurdles','mid']
cols_horses=['rid', 'horseName', 'horseLink', 'age', 'saddle', 'decimalPrice', 'trainerLink', 'trainerName', 'jockeyLink','jockeyName', 'position', 'positionL', 'dist', 'weightSt', 'weightLb', 'overWeight', 'outHandicap', 'headGear', 'RPR', 'TR', 'OR', 'father','mother', 'gfather', 'fatherlink', 'motherlink', 'gfatherlink', 'comment', 'mid', 'rid_win', 'res_win', 'win_mean','win_high','win_low','win_open','win_close']


# Due to possible days mismatch load a day before and a day after
def load_three(d):
    dfm_list=[]
    dfr_list=[]
    dfc_list=[]
    # Prev2 Day
    ds=(d+timedelta(days=-1)).strftime('%Y-%m-%d')
    fn=f'data/BF/markets_{ds}.csv'
    if path.exists(fn):
        dfc_list.append(pd.read_csv(f'data/BF/changes_{ds}.csv', index_col=False))
    # Prev Day
    ds=(d+timedelta(days=-1)).strftime('%Y-%m-%d')
    fn=f'data/BF/markets_{ds}.csv'
    if path.exists(fn):
        dfm_list.append(pd.read_csv(fn))
        dfr_list.append(pd.read_csv(f'data/BF/runners_{ds}.csv', index_col=False))
        dfc_list.append(pd.read_csv(f'data/BF/changes_{ds}.csv', index_col=False))
    # Current Day
    ds=d.strftime('%Y-%m-%d')
    fn=f'data/BF/markets_{ds}.csv'
    if path.exists(fn):
        dfm_list.append(pd.read_csv(fn))
        dfr_list.append(pd.read_csv(f'data/BF/runners_{ds}.csv', index_col=False))
        dfc_list.append(pd.read_csv(f'data/BF/changes_{ds}.csv', index_col=False))
    # Next Day 
    ds=(d+timedelta(days=1)).strftime('%Y-%m-%d')
    fn=f'data/BF/markets_{ds}.csv'
    if path.exists(fn):
        dfm_list.append(pd.read_csv(fn))
        dfr_list.append(pd.read_csv(f'data/BF/runners_{ds}.csv', index_col=False))
        dfc_list.append(pd.read_csv(f'data/BF/changes_{ds}.csv', index_col=False))
    
    allmarkets=pd.concat(dfm_list,ignore_index=True)
    allmarkets=allmarkets[allmarkets['marketType']=='WIN']
    allmarkets['marketTime']=pd.to_datetime(allmarkets['marketTime'])
    allmarkets['marketTime']=allmarkets['marketTime'].dt.tz_localize(None)
    allmarkets['dt']=pd.to_datetime(allmarkets['dt'])
    allmarkets=allmarkets.sort_values(by=['dt'])
    allmarkets=allmarkets.drop_duplicates(subset=['marketType','marketTime','eventName'], keep='last')

    allrunners=pd.concat(dfr_list,ignore_index=True)
    allrunners=allrunners[allrunners['mid'].isin(allmarkets['mid'])]
    allrunners['wname']=allrunners['name'].str.lower().replace(regex=True,to_replace=r'\W',value=r'')
    allrunners['res']=np.where(allrunners['status']=='WINNER',1,0)
    runners=allrunners[allrunners['res']==1]

    allmarkets=allmarkets.merge(runners[['mid','wname']], on='mid', how='left')

    allchanges=pd.concat(dfc_list,ignore_index=True)
    allchanges['dt']=pd.to_datetime(allchanges['dt'])
    allchanges['dt']=allchanges['dt']-timedelta(hours=6)
    allchanges['price']=1/allchanges['price']
    allchanges=allchanges.sort_values(by=['dt'])

    return allmarkets,allrunners,allchanges

def get_changes_stat(x):
    op_mean=op_high=op_low=op_open=op_close=np.NaN
    dfop=changes[(changes['mid']==x[0]) & (changes['rid']==x[1]) & (changes['dt']<x[2]-timedelta(minutes=5))]
    if len(dfop.index)>0:
        op_mean=dfop['price'].mean()
        op_high=dfop['price'].max()
        op_low=dfop['price'].min()
        op_close=dfop.iloc[-1,4]
        op_open=dfop.iloc[0,4]
    return (op_mean,op_high,op_low,op_open,op_close)

stat_cols=['win_mean','win_high','win_low','win_open','win_close']
def save_day(d,races,races_nobind,horses,horses_nobind,changes):
    for col in stat_cols:
        horses_nobind[col]=np.NaN
    races=pd.concat([races,races_nobind], axis=0)
    horses=pd.concat([horses,horses_nobind], axis=0)
    races[cols_races].to_csv('data/done/races_{:%Y-%m-%d}.csv'.format(d),index=False)
    horses[cols_horses].to_csv('data/done/horses_{:%Y-%m-%d}.csv'.format(d),index=False)
    if not changes.empty:
        changes[['mid', 'dt', 'rid', 'price']].to_csv('data/done/changes_{:%Y-%m-%d}.csv'.format(d),index=False)
    return races,horses

# Bind to BF data

In [ ]:
outraces=pd.DataFrame()
outhorses=pd.DataFrame()

year='2015'
ds = datetime.strptime(f'{year}-05-01', '%Y-%m-%d')
de = datetime.strptime(f'{year}-12-31', '%Y-%m-%d')
d = ds
while d <= de:
    print(d)
    races=allraces[allraces['date']==d]
    horses=allhorses[allhorses['rid'].isin(races['rid'])]
    allmarkets,allrunners,allchanges=load_three(d)
    races['runners']=races['rid'].apply(lambda x: len(horses[horses['rid']==x].index))

    #.sort_values(by=['eid','mid','marketTime'])
    
    races=races.merge(allmarkets[['marketTime','countryCode','wname','mid']], on=['marketTime','countryCode','wname'], how='left')
    races['mid']=races['mid'].fillna(0)
    races['mid']=races['mid'].astype(int)

    horses=horses.merge(races[['rid','mid']], on='rid', how='left')
    horses=horses.merge(allrunners[['wname','mid','rid','res']], on=['wname','mid'], how='left')
    horses['mid']=horses['mid'].fillna(0)
    horses['mid']=horses['mid'].astype(int)
    horses['rid_y']=horses['rid_y'].fillna(0)
    horses['rid_y']=horses['rid_y'].astype(int)
    horses.rename(columns={'rid_x':'rid','rid_y':'rid_win','res':'res_win'}, inplace=True)

    # Remove races & horses w/o odds
    races_noodds_ids=horses[horses['decimalPrice'].isna()]['rid'].unique()
    horses=horses[~horses['decimalPrice'].isna()]
    races=races[~races['rid'].isin(races_noodds_ids)]
    if len(horses.index)==0:
        d += timedelta(days=1)
        continue

    # Split bend and not
    races_nobind=races[races['mid']==0]
    horses_nobind=horses[horses['mid']==0]
    races=races[races['mid']>0]
    horses=horses[horses['mid']>0]

    # calc changes stats for bend only
    if len(horses.index)>0:
        changes=allchanges[(allchanges['mid'].isin(horses['mid'])) & (allchanges['rid'].isin(horses['rid_win']))]
        horses[['win_mean','win_high','win_low','win_open','win_close']] = horses.apply(lambda x: get_changes_stat(x[['mid','rid_win','marketTime']]), axis=1, result_type='expand')
    else:
        changes=pd.DataFrame()
    # save a day for a site
    races,horses=save_day(d,races,races_nobind,horses,horses_nobind,changes)

    # add to big table for an analysis
    if outraces.empty:
        outraces=races
    else:
        outraces=pd.concat([outraces,races],ignore_index=True)

    if outhorses.empty:
        outhorses=horses
    else:
        outhorses=pd.concat([outhorses,horses],ignore_index=True)

    d += timedelta(days=1)
outraces=outraces.drop_duplicates(subset=['rid'], keep='last')
outhorses=outhorses.drop_duplicates(subset=['rid','horseName'], keep='last')
outraces[cols_races].to_csv(f'data/races_binded_{year}.csv', index=False)
outhorses[cols_horses].to_csv(f'data/horses_binded_{year}.csv', index=False)

# Probs

In [ ]:
def horses_count(id):
    return len(allhorses[allhorses['rid']==id].index)

def get_changes_stat(x):
    op_mean=op_high=op_low=op_start=op_close=np.NaN
    dfop=allchanges[(allchanges['mid']==x[0]) & (allchanges['rid']==x[1]) & (allchanges['dt']<x[2]-timedelta(minutes=5))]
    if len(df.index)>0:
        op_mean=dfop['price'].mean()
        op_high=dfop['price'].max()
        op_low=dfop['price'].min()
        op_close=dfop.iloc[-1,3]
        op_open=dfop.iloc[0,3]
    return op_mean,op_high,op_low,op_open,op_close

year=2020

allraces=pd.read_csv(f'data/races_binded_{year}.csv', index_col=False)
allraces['marketTime']=pd.to_datetime(allraces['marketTime'])
allraces['marketTime']=allraces['marketTime'].dt.tz_localize(None)
allhorses=pd.read_csv(f'data/horses_binded_{year}.csv', index_col=False)
allhorses=allhorses.merge(allraces[['rid','marketTime']], on='rid', how='left')
allchanges=pd.read_csv(f'data/changes_binded_{year}.csv', index_col=False)
allchanges['dt']=pd.to_datetime(allchanges['dt'])

allchanges=allchanges[(allchanges['mid'].isin(allraces['mid_win'])) | (allchanges['mid'].isin(allraces['mid_place']))]
allchanges['price']=1/allchanges['price']
allchanges=allchanges.reset_index(drop=True)
allchanges=allchanges.sort_values(by=['dt'])
allraces['runners']=allraces['rid'].apply(lambda x: horses_count(x))

allhorses[['win_mean','win_high','win_low','win_open','win_close']]= df.apply(lambda x: get_changes(x[['mid_win','rid_win','marketTime']]), axis=1, result_type='expand')
allhorses[['place_mean','place_high','place_low','place_open','place_close']]= df.apply(lambda x: get_changes(x[['mid_place','rid_place','marketTime']]), axis=1, result_type='expand')

process_year(2020)

In [ ]:
allhorses=pd.read_csv(f'data/horses_binded_{year}.csv', index_col=False)
allraces=pd.read_csv(f'data/races_binded_{year}.csv', index_col=False)
allraces['marketTime']=pd.to_datetime(allraces['marketTime'])
allraces['marketTime']=allraces['marketTime'].dt.tz_localize(None)
allhorses=allhorses.merge(allraces[['rid','marketTime']], on='rid', how='left')

In [ ]:
allraces['date'].max()

In [ ]:
allraces['country']=allraces['course'].apply(lambda x: x.split('(')[-1].replace(')',''))
allraces['country']=np.where(allraces['country']=='AW','GB',allraces['country'])
allraces['country']=np.where(allraces['country'].str.len()>4,'GB',allraces['country'])

In [ ]:
allraces['country']=np.where(allraces['country']=='IRE','IE',allraces['country'])
allraces['country']=np.where(allraces['country']=='USA','US',allraces['country'])
allraces['country']=np.where(allraces['country']=='AUS','AU',allraces['country'])
allraces['country']=np.where(allraces['country']=='Ayr','GB',allraces['country'])
allraces['country']=np.where(allraces['country']=='JPN','JP',allraces['country'])

In [ ]:
def parse_dt(x):
    h,m=map(int,x[0].split(':'))
    if h<9:
        h+=12
    ds = datetime.strptime(x[1], '%Y-%m-%d')
    ds+= timedelta(hours=h)
    ds+= timedelta(minutes=m)
    return ds

allraces.apply(lambda x: parse_dt(x[['time','date']]), axis=1)

# Remove doubles

In [ ]:
d = datetime.strptime('2018-12-12', '%Y-%m-%d')

runners=pd.read_csv('data/BF/runners_{:%Y-%m-%d}.csv'.format(d))
changes=pd.read_csv('data/BF/changes_{:%Y-%m-%d}.csv'.format(d))

In [ ]:
d='2020-04'
files = glob.glob(f'data/BF/markets_{d}*.csv')
for f in files:
    df=pd.read_csv(f)
    df=df[df['marketType'].isin(['WIN'])].sort_values(by=['eid','mid','marketTime'])
    df=df.drop_duplicates(subset=['mid'], keep='last')
    df=df.reset_index(drop=True)
    df.to_csv(f.replace('markets','done/markets'))
    #break

In [ ]:
files = glob.glob(f'data/BF/runners_{d}*.csv')
for f in files:
    print(f)
    df=pd.read_csv(f)
    df=df.drop_duplicates(subset=['mid','rid'], keep='last')
    df['name'].replace(regex=True,inplace=True,to_replace=r'\d+\.\s',value=r'')
    df=df[~(df['status']=='REMOVED')]
    df=df.reset_index(drop=True)
    df[['mid','dt','rid','name','sortPriority','adjustmentFactor','status']].to_csv(f.replace('runners','done/runners'))
    #break


In [ ]:
files = glob.glob(f'data/BF/changes_{d}*.csv')
for f in files:
    print(f)
    df=pd.read_csv(f)
    df=df.drop_duplicates(subset=['mid','rid','dt'], keep='last')
    df=df.reset_index(drop=True)
    df.to_csv(f.replace('changes','done/changes'))
    #break

## Keep only necessary runners and changes

In [ ]:
d='2020-04'
files = glob.glob(f'data/BF/markets_{d}*.csv')
for f in files:
    print(f[-14:])
    dfm=pd.read_csv(f)
    dfr=pd.read_csv('data/BF/runners_'+f[-14:])
    dfc=pd.read_csv('data/BF/changes_'+f[-14:])
    
    dfr=dfr[dfr['mid'].isin(dfm['mid'])].reset_index(drop=True)
    dfr=dfr.drop(columns=['Unnamed: 0'])
    dfc=dfc[(dfc['mid'].isin(dfr['mid'])) & (dfc['rid'].isin(dfr['rid']))]
    dfc=dfc.drop(columns=['Unnamed: 0'])

    dfr.to_csv('data/BF/done/runners_'+f[-14:])
    dfc.to_csv('data/BF/done/changes_'+f[-14:])
    #break

# Split on two by date (May 1st, 2015) 
## due to betfair data starts from April 30

horses=pd.read_csv('data/horses_all.csv')
horses1=pd.read_csv('data/horses.csv')
horses1.columns=horses.columns
horses.to_csv('data/horses_all.csv')

In [ ]:
horses=pd.read_csv('data/horses_all.csv', index_col=False)
allraces=pd.read_csv('data/races_all.csv', index_col=False)
allraces['date']=pd.to_datetime(allraces['date'])


In [ ]:
ds = datetime.strptime('2015-04-30', '%Y-%m-%d')
races=allraces[allraces['date']>ds]
races.reset_index(drop=True)
races.to_csv('data/races_bf.csv')

hrs=horses[horses['rid'].isin(races['rid'])]
hrs.reset_index(drop=True)
hrs.to_csv('data/horses_bf.csv')

races=allraces[allraces['date']<=ds]
races.reset_index(drop=True)
races.to_csv('data/races_nobf.csv')

hrs=horses[horses['rid'].isin(races['rid'])]
hrs.reset_index(drop=True)
hrs.to_csv('data/horses_nobf.csv')